In [158]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [143]:
DF = pd.read_csv('data_cleaned/MERGED_PETER.csv')

DF = DF[~pd.isna(DF['Population'])]
DF = DF[~pd.isna(DF['Area_(sqmi)'])].reset_index(drop=True)
DF.dtypes

Unique_ID                               object
date                                    object
county                                  object
state                                   object
fips                                   float64
cases                                    int64
deaths                                   int64
Latitude                               float64
Longitude                              float64
Total_Hospital_Beds                     object
Total_ICU_Beds                          object
Available_Hospital_Beds                 object
Potentially_Available_Hospital_Beds     object
Available_ICU_Beds                     float64
Potentially_Available_ICU_Beds          object
Adult_Population                        object
Population_65+                          object
Area_(sqmi)                            float64
Population                             float64
dtype: object

In [144]:
for col in DF.columns[5:] :
    DF[col] = DF[col].apply(lambda x : float(str(x).replace(',','')))
DF.dtypes

Unique_ID                               object
date                                    object
county                                  object
state                                   object
fips                                   float64
cases                                  float64
deaths                                 float64
Latitude                               float64
Longitude                              float64
Total_Hospital_Beds                    float64
Total_ICU_Beds                         float64
Available_Hospital_Beds                float64
Potentially_Available_Hospital_Beds    float64
Available_ICU_Beds                     float64
Potentially_Available_ICU_Beds         float64
Adult_Population                       float64
Population_65+                         float64
Area_(sqmi)                            float64
Population                             float64
dtype: object

In [145]:
hospital_ids = np.unique(DF.loc[~pd.isna(DF['Total_Hospital_Beds']),['Unique_ID']].values)

item_seen = []
hospital_locations = []
for i,v in DF.iterrows() :
    if v['Unique_ID'] in hospital_ids :
        if v['Unique_ID'] not in item_seen :
            item_seen.append(v['Unique_ID'])
            hospital_locations.append([v['Latitude'],v['Longitude']])


In [146]:
DF['Pop_Density'] = DF['Population']/DF['Area_(sqmi)']

In [153]:
big_city_ids = set(DF[DF['Pop_Density'] >= 3000]['Unique_ID'])

item_seen = []
big_city_locations = []
for i,v in DF.iterrows() :
    if v['Unique_ID'] in big_city_ids :
        if v['Unique_ID'] not in item_seen :
            item_seen.append(v['Unique_ID'])
            big_city_locations.append([v['Latitude'],v['Longitude']])


In [173]:
def Proximity(location_x,location_y,city_loc,hospital_loc) :
    
    min_distance_city = 1e10
    for loc in city_loc :
        dis = np.sqrt((loc[0] - location_x)**2 + (loc[1] - location_y)**2)
        if dis < min_distance_city :
            min_distance_city = dis
            
    min_distance_hosp = 1e10
    for loc in hospital_loc :
        dis = np.sqrt((loc[0] - location_x)**2 + (loc[1] - location_y)**2)
        if dis < min_distance_hosp :
            min_distance_hosp = dis
            
    return min_distance_city,min_distance_hosp

In [177]:
nearest_big_city,nearest_hospital = [],[]
for i,v in DF.iterrows() :
    if i%500 == 0 :print(i)
    cit,hos = Proximity(v['Latitude'],v['Longitude'],big_city_locations,hospital_locations)
    nearest_big_city.append(cit)
    nearest_hospital.append(hos)

DF['Nearest_Hospital'] = nearest_hospital
DF['Nearest_BigCity'] = nearest_big_city

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500


In [179]:
DF.to_csv('data_cleaned/Distance_Calculations.csv',index=False)